In [1]:
import xarray as xr
from Algorithms.fetchfields import *

In [2]:
ds = xr.open_dataset("http://thredds.met.no/thredds/dodsC/meps25epsarchive/2017/08/27/meps_subset_2_5km_20170827T00Z.nc")

In [3]:
nx,ny,geo_pl, geo_sf, airtemp_pl, rhs_pl, lowcloud, prec = fetchfields(ds,0)

In [4]:
from Algorithms.HelicopterTriggerIndex import *

In [6]:
Temperature = temperature_max_band_from_b_to_c(5,-5)
f= Temperature
x = np.linspace(-6,6,100,dtype=float)
    

In [7]:
HTI = HelicopterTriggerIndex([f])

In [15]:
HTI(airtemp_pl.sel(pressure=925))


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])